In [1]:
# Fourier, Characteritic clabiration of Heston model


In [2]:
# Import modules 
import math
import numpy as np
from scipy.stats import norm
import scipy.optimize as optimize
import matplotlib.pyplot as plt

In [8]:
# Market information, parameters, we want to find values for kappa, theta, rho...?
# Market, option,share dynamics and information
r     = 0.06 # continously compounded risk free rate
S_0   = 100  # initial stock price
sigma = 0.5  # constant volatility of the share price

# Stochastic volatility parameters
v_0   = 0.06
#kappa = None, values must be estimated
#theta = None, values must be estimated
#sigma_v = None, value must be estimated
rho   = -0.4

# Market option specific information
K     = np.array([110,100,90])  # Strike price of call option
price = [8.02,12.63,18.72]
k_log = np.log(K)
k_log.shape = (3,1)
T     = 1   # Time until maturity 

#Approximation information
# Initialisin and calculating parameters for estimation t_max, N, t_delta for Gil Pelaez
t_max    = 30
N        = 100
delta_t  = t_max/N
# The array of various t_k points
from_1_N = np.linspace(1,N,N)
t_k      = (from_1_N-1/2)*delta_t  

In [9]:
# Characteristic function code
# Characteristic functon of sT = log(ST) => exp(C(T;u) + D(T,u)*v_t + iu*log(S_t))  # i complex number
# C(T,u) = riTu + theta*x[Tx - 1/a*log(1-gexp(d*r)/1-g)]
# D(T,u) = [(1-exp(d*r))/1-gexp(d*r)]*x_
#  x_    = b-d/2a
# x+     = b+d/2a
# c      = (-u**2+ui)/2
# g      = x_/x+
# d      = sqrt(b**2-4ac)
# b      = kappa - rho*sigma*i*u
# a      = sigma**2/2

a   = sigma**2/2

def b(u):
    return kappa - rho*sigma*1j*u

def c(u):
    return -(u**2 + u*1j)/2

def d(u):
    return np.sqrt(b(u)**2 - 4*a*c(u))

def x_minus(u):
    return (b(u)-d(u))/(2*a)

def x_plus(u):
    return (b(u)+d(u))/(2*a)

def g(u):
    return x_minus(u)/x_plus(u)

def C(u):
    value = T*x_minus(u) - np.log((1-g(u)*np.exp(d(u)*-T))/(1-g(u)))/a
    return r*1j*T*u + theta*kappa*value

def D(u):
    value1 = 1-np.exp(d(u)*-T)
    value2 = 1-g(u)*np.exp(d(u)*-T)
    return (value1/value2)*x_minus(u)

def log_char(u):  #char_M1
    return np.exp(C(u) + D(u)*v_0 + 1j*u*np.log(S_0))
    
# Change of measure, converting charactersitic function directly instead of direct change of measure
# Risk neutral measure under Q_S so we can apply Gil Pelaez
# char_S(u) = char(u-i) / char(-i)
    
def adj_char(u):    #char_M2
    return log_char(u-1j) / log_char(-1j)   

In [10]:
# Calibration functions
def heston_pricer(array_estimates):
    theta = array_estimates[0]
    kappa    = array_estimates[1]
    sigma = array_estimates[2]
    first_intergral   = sum((((np.exp(-1j*t_k*k_log))*adj_char(t_k)).imag/t_k)*delta_t)
    second_intergral  = sum((((np.exp(-1j*t_k*k_log))*log_char(t_k)).imag/t_k)*delta_t)
    return S_0*(1/2 + first_intergral/np.pi) - np.exp(-r*T)*K*(1/2 + second_intergral/np.pi)


def optimizer(array_estimates):    
    return  sum(np.abs(price- heston_pricer(array_estimates)))

In [11]:
# Find optimal values using initial estimates

optimal_values = optimize.fmin_slsqp(optimizer,(0.1,3,0.1))

TypeError: unsupported operand type(s) for -: 'NoneType' and 'complex'